# Imports and Data Setup Code (from Spec Sheet)

In [2]:
from folktables import folktables
from folktables import ACSDataSource
import numpy as np

# (Age) must be greater than 16 and less than 90,
# and (Person weight) must be greater than or equal to 1
def employment_filter(data):
    """Filters for the employment prediction task"""
    df = data
    df = df[df['AGEP'] > 16]
    df = df[df['AGEP'] < 90]
    df = df[df['PWGTP'] >= 1]
    return df

ACSEmployment = folktables.BasicProblem(
    features=[
        'AGEP',  # age; for range of values of features please check Appendix B.4 of Retiring Adult: New Datasets for Fair Machine Learning NeurIPS 2021 paper
        'SCHL',  # educational attainment
        'MAR',   # marital status
        'RELP',  # relationship
        'DIS',   # disability recode
        'ESP',   # employment status of parents
        'CIT',   # citizenship status
        'MIG',   # mobility status (lived here 1 year ago)
        'MIL',   # military service
        'ANC',   # ancestry recode
        'NATIVITY',  # nativity
        'DEAR',   # hearing difficulty
        'DEYE',   # vision difficulty
        'DREM',   # cognitive difficulty
        'SEX',    # sex
        'RAC1P',  # recoded detailed race code
        'GCL',    # grandparents living with grandchildren
    ],
    target='ESR',  # employment status recode
    target_transform=lambda x: x == 1,
    group='DIS',
    preprocess=employment_filter,
    postprocess=lambda x: np.nan_to_num(x, -1),
)

data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["FL"], download=True)  # data for Florida state

features, label, group = ACSEmployment.df_to_numpy(acs_data)

from aif360.datasets import StandardDataset
import pandas as pd
import numpy as np

data = pd.DataFrame(features, columns=ACSEmployment.features)
data['label'] = label

favorable_classes = [True]
protected_attribute_names = [ACSEmployment.group]
privileged_classes = np.array([[1]])

data_for_aif = StandardDataset(
    data,
    label_name='label',
    favorable_classes=favorable_classes,
    protected_attribute_names=protected_attribute_names,
    privileged_classes=privileged_classes
)

privileged_groups = [{'DIS': 1}]
unprivileged_groups = [{'DIS': 2}]

pip install 'aif360[LawSchoolGPA]'
/Users/kyhi2018/Desktop/Imperial/Ethics, Fairness, and Explainability in AI/CW1 - Fairness/Fairness-in-AI/venv/lib/python3.11/site-packages/aif360/datasets/standard_dataset.py:143: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[pos, label_name] = favorable_label


# Task 1

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from aif360.metrics import ClassificationMetric
import pandas as pd
import joblib

# Split the dataset into train-val and test sets
train_and_val_data, test_data = data_for_aif.split([0.7], shuffle=True, seed=0)

# Set up loop for hyperparameter tuning/exploration (C, regularisation strength)
results = pd.DataFrame(columns=['C', 'Solver','Mean accuracy', 'Mean EOD'])

best_accuracy = 0 # initialize best accuracy
best_eod = 1 # initialize best EOD

for C in [ 10**i for i in range(-8, 1, 1)]: # loop through different C values
    for solver in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']: # loop through different solvers
        print("Training model with C =", C, "and solver =", solver)
        
        # Initialise model
        model = LogisticRegression(solver=solver, C=C, random_state=0)

        # Set up loop for different train-val splits
        accuracy_list = [] # initialize list to store accuracy values
        eod_list = [] # initialize list to store EOD values

        for seed_index in [10*i for i in range(5)]:
            # Split train-val set into train and val sets
            train_data, val_data = train_and_val_data.split([0.8], shuffle=True, seed=seed_index)

            # Normalize the train and val datasets
            scale_orig = StandardScaler()
            x_train = scale_orig.fit_transform(train_data.features)
            y_train = train_data.labels.ravel()
            x_val = scale_orig.transform(val_data.features)
            y_val = val_data.labels.ravel()

            # Model training and prediction
            model.fit(x_train,y_train)
            predictions = model.predict(x_val)
            val_pred = val_data.copy()
            val_pred.labels = predictions

            # Metrics
            metrics = ClassificationMetric(val_data, val_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
            eod = (metrics.equal_opportunity_difference()) # Equal opportunity difference
            
            # Save model if it has the best accuracy or EOD
            if metrics.accuracy() > best_accuracy:
                joblib.dump(model, 'std_model_accuracy.joblib')
                best_accuracy = metrics.accuracy()
            if abs(metrics.equal_opportunity_difference()) < best_eod:
                joblib.dump(model, 'std_model_eod.joblib')
                best_eod = abs(metrics.equal_opportunity_difference())
            
            # Store accuracy and EOD values
            accuracy_list.append(metrics.accuracy())
            eod_list.append(eod)
        
        # Calculate mean accuracy and EOD values and append to lists
        mean_accuracy = np.mean(accuracy_list)
        mean_eod = np.mean(eod_list)
        
        # Append results to dataframe
        new_result  = pd.DataFrame([[C, solver, mean_accuracy, mean_eod]], columns=['C', 'Solver','Mean accuracy', 'Mean EOD'])
        results = pd.concat([results, new_result], ignore_index=True)

Training model with C = 1e-08 and solver = newton-cg


/var/folders/3v/n0qzq7z56gj9ts7dnzkhpbr80000gn/T/ipykernel_54783/536532560.py:66: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, new_result], ignore_index=True)


Training model with C = 1e-08 and solver = lbfgs
Training model with C = 1e-08 and solver = liblinear
Training model with C = 1e-08 and solver = sag
Training model with C = 1e-08 and solver = saga
Training model with C = 1e-07 and solver = newton-cg
Training model with C = 1e-07 and solver = lbfgs
Training model with C = 1e-07 and solver = liblinear
Training model with C = 1e-07 and solver = sag
Training model with C = 1e-07 and solver = saga
Training model with C = 1e-06 and solver = newton-cg
Training model with C = 1e-06 and solver = lbfgs
Training model with C = 1e-06 and solver = liblinear
Training model with C = 1e-06 and solver = sag
Training model with C = 1e-06 and solver = saga
Training model with C = 1e-05 and solver = newton-cg
Training model with C = 1e-05 and solver = lbfgs
Training model with C = 1e-05 and solver = liblinear
Training model with C = 1e-05 and solver = sag
Training model with C = 1e-05 and solver = saga
Training model with C = 0.0001 and solver = newton-cg

### Training and Validation Results

In [75]:
# Find highest accuracy and lowest EOD
highest_accuracy = results['Mean accuracy'].max()
lowest_eod = results['Mean EOD'].min() 

# Find lowest EOD that is not 0
lowest_nonzero_eod = results.loc[results['Mean EOD'] != 0]['Mean EOD'].abs().min()

# Find the corresponding C and solver values
best_accuracy = results.loc[results['Mean accuracy'] == highest_accuracy]
best_eod = results.loc[results['Mean EOD'] == lowest_eod]
best_nonzero_eod = results.loc[(results['Mean EOD'] == lowest_nonzero_eod) | (results['Mean EOD'] == -lowest_nonzero_eod)]

print(f"Best accuracy:\n", best_accuracy, f'\n')
print(f"Best EOD:\n", best_eod, f'\n')
print(f"Lowest non-zero EOD:\n", best_nonzero_eod, f'\n')

Best accuracy:
         C     Solver  Mean accuracy  Mean EOD
25  0.001  newton-cg        0.72007 -0.018474
29  0.001       saga        0.72007 -0.018457 

Best EOD:
        C Solver  Mean accuracy  Mean EOD
34  0.01   saga       0.719925 -0.022143 

Lowest non-zero EOD:
         C Solver  Mean accuracy  Mean EOD
28  0.001    sag       0.720036 -0.018224 



### Test Results

In [18]:
# Load the best model
model_accuracy = joblib.load('std_model_accuracy.joblib')
model_eod = joblib.load('std_model_eod.joblib')

# Normalize the test dataset
scale_orig = StandardScaler()
x_test = scale_orig.fit_transform(test_data.features)
y_test = test_data.labels.ravel()

# Model prediction
predictions_accuracy = model_accuracy.predict(x_test)
predictions_eod = model_eod.predict(x_test)

# Create test dataset with predictions
test_pred_accuracy = test_data.copy()
test_pred_accuracy.labels = predictions_accuracy
test_pred_eod = test_data.copy()
test_pred_eod.labels = predictions_eod

# Metrics
metrics_best_accuracy = ClassificationMetric(test_data, test_pred_accuracy, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
best_accuracy_accuracy = metrics_best_accuracy.accuracy()
best_accuracy_eod = metrics_best_accuracy.equal_opportunity_difference()

metrics_best_eod = ClassificationMetric(test_data, test_pred_eod, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
best_eod_accuracy = metrics_best_eod.accuracy()
best_eod_eod = metrics_best_eod.equal_opportunity_difference()

# Print results
print("Most accurate model:")
print(f"Accuracy: {best_accuracy_accuracy}")
print(f"EOD: {best_accuracy_eod}", f'\n')
print("Best EOD model:")
print(f"Accuracy: {best_eod_accuracy}")
print(f"EOD: {best_eod_eod}")


Most accurate model:
Accuracy: 0.7523310487727127
EOD: 0.6736529561484942 

Best EOD model:
Accuracy: 0.525880618425247
EOD: 0.0


# Task 2

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from aif360.metrics import ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
import pandas as pd
import joblib

# Split the dataset into train-val and test sets
train_and_val_data, test_data = data_for_aif.split([0.7], shuffle=True, seed=0)

# Set up loop for hyperparameter tuning/exploration (C, regularisation strength)
results = pd.DataFrame(columns=['C', 'Solver','Mean accuracy', 'Mean EOD'])

best_accuracy = 0 # initialize best accuracy
best_eod = 1 # initialize best EOD

for C in [ 10**i for i in range(-8, 1, 1)]: # loop through different C values
    for solver in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']: # loop through different solvers
        print("Training model with C =", C, "and solver =", solver)
        
        # Initialise model
        model = LogisticRegression(solver=solver, C=C, random_state=0)

        # Set up loop for different train-val splits
        accuracy_list = [] # initialize list to store accuracy values
        eod_list = [] # initialize list to store EOD values

        for seed_index in [10*i for i in range(5)]:
            # Split train-val set into train and val sets
            train_data, val_data = train_and_val_data.split([0.8], shuffle=True, seed=seed_index)

            # Transform the original dataset via reweighing
            RW = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
            train_data = RW.fit_transform(train_data)
            
            # Normalize the train and val datasets
            scale_orig = StandardScaler()
            x_train = scale_orig.fit_transform(train_data.features)
            y_train = train_data.labels.ravel()
            x_val = scale_orig.transform(val_data.features)
            y_val = val_data.labels.ravel()
            
            # Model training and prediction
            model.fit(x_train, y_train, sample_weight=train_data.instance_weights)
            predictions = model.predict(x_val)
            val_pred = val_data.copy()
            val_pred.labels = predictions

            # Metrics
            metrics = ClassificationMetric(val_data, val_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
            eod = (metrics.equal_opportunity_difference()) # Equal opportunity difference
            
            # Save model if it has the best accuracy or EOD
            if metrics.accuracy() > best_accuracy:
                joblib.dump(model, 'fair_model_accuracy.joblib')
                best_accuracy = metrics.accuracy()
            if abs(metrics.equal_opportunity_difference()) < best_eod:
                joblib.dump(model, 'fair_model_eod.joblib')
                best_eod = abs(metrics.equal_opportunity_difference())
            
            # Store accuracy and EOD values
            accuracy_list.append(metrics.accuracy())
            eod_list.append(eod)
        
        # Calculate mean accuracy and EOD values and append to lists
        mean_accuracy = np.mean(accuracy_list)
        mean_eod = np.mean(eod_list)
        
        # Append results to dataframe
        new_result  = pd.DataFrame([[C, solver, mean_accuracy, mean_eod]], columns=['C', 'Solver','Mean accuracy', 'Mean EOD'])
        results = pd.concat([results, new_result], ignore_index=True)

Training model with C = 1e-08 and solver = newton-cg


/var/folders/3v/n0qzq7z56gj9ts7dnzkhpbr80000gn/T/ipykernel_54783/2889587750.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, new_result], ignore_index=True)


Training model with C = 1e-08 and solver = lbfgs
Training model with C = 1e-08 and solver = liblinear
Training model with C = 1e-08 and solver = sag
Training model with C = 1e-08 and solver = saga
Training model with C = 1e-07 and solver = newton-cg
Training model with C = 1e-07 and solver = lbfgs
Training model with C = 1e-07 and solver = liblinear
Training model with C = 1e-07 and solver = sag
Training model with C = 1e-07 and solver = saga
Training model with C = 1e-06 and solver = newton-cg
Training model with C = 1e-06 and solver = lbfgs
Training model with C = 1e-06 and solver = liblinear
Training model with C = 1e-06 and solver = sag
Training model with C = 1e-06 and solver = saga
Training model with C = 1e-05 and solver = newton-cg
Training model with C = 1e-05 and solver = lbfgs
Training model with C = 1e-05 and solver = liblinear
Training model with C = 1e-05 and solver = sag
Training model with C = 1e-05 and solver = saga
Training model with C = 0.0001 and solver = newton-cg

### Results Analysis

In [74]:
# find highest accuracy and lowest EOD
highest_accuracy = results['Mean accuracy'].max()
lowest_eod = results['Mean EOD'].min() 

# find lowest EOD that is not 0
lowest_nonzero_eod = results.loc[results['Mean EOD'] != 0]['Mean EOD'].abs().min()

# find the corresponding C and solver values
best_accuracy = results.loc[results['Mean accuracy'] == highest_accuracy]
best_eod = results.loc[results['Mean EOD'] == lowest_eod]
best_nonzero_eod = results.loc[(results['Mean EOD'] == lowest_nonzero_eod) | (results['Mean EOD'] == -lowest_nonzero_eod)]

print(f"Best accuracy:\n", best_accuracy, f'\n')
print(f"Best EOD:\n", best_eod, f'\n')
print(f"Lowest non-zero EOD:\n", best_nonzero_eod, f'\n')

Best accuracy:
         C     Solver  Mean accuracy  Mean EOD
25  0.001  newton-cg        0.72007 -0.018474
29  0.001       saga        0.72007 -0.018457 

Best EOD:
        C Solver  Mean accuracy  Mean EOD
34  0.01   saga       0.719925 -0.022143 

Lowest non-zero EOD:
         C Solver  Mean accuracy  Mean EOD
28  0.001    sag       0.720036 -0.018224 



### Test Results

In [22]:
# Load the best model
model_accuracy = joblib.load('fair_model_accuracy.joblib')
model_eod = joblib.load('fair_model_eod.joblib')

# Normalize the test dataset
scale_orig = StandardScaler()
x_test = scale_orig.fit_transform(test_data.features)
y_test = test_data.labels.ravel()

# Model prediction
predictions_accuracy = model_accuracy.predict(x_test)
predictions_eod = model_eod.predict(x_test)

# Create test dataset with predictions
test_pred_accuracy = test_data.copy()
test_pred_accuracy.labels = predictions_accuracy
test_pred_eod = test_data.copy()
test_pred_eod.labels = predictions_eod

# Metrics
metrics_best_accuracy = ClassificationMetric(test_data, test_pred_accuracy, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
best_accuracy_accuracy = metrics_best_accuracy.accuracy()
best_accuracy_eod = metrics_best_accuracy.equal_opportunity_difference()

metrics_best_eod = ClassificationMetric(test_data, test_pred_eod, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
best_eod_accuracy = metrics_best_eod.accuracy()
best_eod_eod = metrics_best_eod.equal_opportunity_difference()

# Print results
print("Most accurate model:")
print(f"Accuracy: {best_accuracy_accuracy}")
print(f"EOD: {best_accuracy_eod}", f'\n')
print("Best EOD model:")
print(f"Accuracy: {best_eod_accuracy}")
print(f"EOD: {best_eod_eod}")

Most accurate model:
Accuracy: 0.7197760599298693
EOD: 0.004663503874752228 

Best EOD model:
Accuracy: 0.525880618425247
EOD: 0.0


# Task 3